In [1]:
#import osmnx as ox
#import geopy.distance as dst
#import matplotlib.pyplot as plt
#import numpy as np

In [2]:
#place = "Santiago de Surco, Lima, Peru"
#G = ox.graph_from_place(place, network_type = 'drive')
#def calcLength(coords_1, coords_2):
#    return dst.geodesic(coords_1, coords_2).m

In [3]:
import geopy.distance as dst
import math
import random as r
import time

In [4]:
def calcLength(coords_1, coords_2):
    return dst.geodesic(coords_1, coords_2).m

def writeNodes(path:str, nodes):
    line = ""
    for nd, nd_dt in enumerate(nodes):
        line += "\n" + str(nd) + " " + str(nd_dt[0]) + " " + str(nd_dt[1])
    file = open(path, "w+")
    file.write(line)
    file.close()

def writeAdjList(path:str, adj_lst):
    line = ""
    for nd, adj_nodes in enumerate(adj_lst):
        line += "\n" + str(nd)
        for adj_nd in adj_nodes:
            adj_nd_dt = adj_nodes[adj_nd]
            line += " " + str(adj_nd) + " " + str(adj_nd_dt[0]) + " " + str(adj_nd_dt[1])
    file = open(path, "w+")
    file.write(line)
    file.close()

def getDataNodes(ntx, path_nodes:str, path_adj_list:str):
    id2indx = {}
    n = len(ntx.nodes.data(True))
    nodes = []
    adj_lst = []
    for i, ntx_nd in enumerate(ntx.nodes.data(True)):
        id2indx[ntx_nd[0]] = i
        nodes.append((ntx_nd[1]['y'], ntx_nd[1]['x']))
        adj_lst.append({})
    for nd_id in ntx:
        nd = id2indx[nd_id]
        for adj_id in ntx[nd_id]:
            adj_nd = id2indx[adj_id]
            lngth = calcLength(nodes[nd], nodes[adj_nd])
            adj_lst[nd][adj_nd] = [lngth, r.randint(0, 100) / 100]
    writeNodes(path_nodes, nodes)
    writeAdjList(path_adj_list, adj_lst)

def only_path(root, end, path):
    if path[end] == -1:return []
    n_path = [end]
    while n_path[-1] != root:
        n_path.append(path[n_path[-1]])
    return list(reversed(n_path))

def getCoefTrafic(hora):
    trafic_o = 0.6
    trafic_base = math.e/(math.sqrt(2 * math.pi) * trafic_o)
    hora = hora % 12
    coef = -0.5 * ((hora - 7.)/ trafic_o / 2 )**2
    value = trafic_base ** coef
    return value

In [5]:
#getDataNodes(G, "dataset/nodes.txt", "dataset/edges.txt")

In [6]:
#!pip install tkintermapview
#!pip install pyperclip

In [7]:
class Graph:
    def __init__(self, nodes_path = "dataset/nodes.txt", edges_path = "dataset/edges.txt"):
        self.nodes = []
        self.adj_lst = []
        if len(nodes_path) > 2 and len(edges_path) > 2:
            file = open(nodes_path, 'r')
            next(file)
            for line in file:
                words = line.split()
                self.nodes.append((float(words[1]), float(words[2])))
                self.adj_lst.append({})
            file.close()
            file = open(edges_path, 'r')
            next(file)
            for line in file:
                words = line.split()
                nd = int(words[0])
                for adj_nd in range(1, len(words), 3):
                    self.adj_lst[nd][int(words[adj_nd])] = ((float(words[adj_nd + 1]), float(words[adj_nd + 2])))
            file.close()
            self.n = len(self.nodes)
            print("nodes:", len(self.nodes))

    def __getitem__(self, node):
        return self.nodes[node] if node in range(len(self.nodes)) else (0,0)

    def edge(self, nd, adj_nd):
        return self.adj_lst[nd][adj_nd]

    def generateHeuristic(self, end):
        end_data = self.nodes[end]
        return [calcLength(nd_dt, end_data) for nd_dt in self.nodes]

    def getWayNodes(self, root, end, hour = 7):
        path, init_t, process_t = self.aStar(root, end, hour)
        return list(only_path(root, end, path)), init_t, process_t

    def getWayNodes2(self, root, end, hour = 7):
        path, init_t, process_t = self.dijkstra(root, end, hour)
        return list(only_path(root, end, path)), init_t, process_t

    def getWayCoords(self, root, end, hour = 7):
        nodes_way, init_t, process_t = self.getWayNodes(root, end, hour)
        n = len(nodes_way)
        for id in range(n):
            nodes_way[id] = self.nodes[nodes_way[id]]
        return nodes_way, init_t, process_t

    def getWayCoords2(self, root, end, hour = 7):
        nodes_way, init_t, process_t = self.getWayNodes2(root, end, hour)
        n = len(nodes_way)
        for id in range(n):
            nodes_way[id] = self.nodes[nodes_way[id]]
        return nodes_way, init_t, process_t

    def normalizePositions(self, pi, pf):
        if pf[0] < pi[0]:
            pi[0], pf[0] = pf[0], pi[0]
        if pf[1] < pi[1]:
            pi[1], pf[1] = pf[1], pi[1]
        
    def getAreaNodes(self, pi, pf, nodes = None):
        area_nodes = []
        if nodes is None:
            nodes = range(len(self.nodes))
        self.normalizePositions(pi, pf)
        for nd in nodes:
            nd_data = self.nodes[nd]
            if nd_data[0] < pi[0] or nd_data[0] > pf[0] or nd_data[1] < pi[1] or nd_data[1] > pf[1]:
                continue
            area_nodes.append(nd)
        return area_nodes

    def getNearestNodes(self, pos, n):
        a_nodes = self.getAreaNodes([pos[0] - n, pos[1] - n], [pos[0] + n, pos[1] + n])
        if len(a_nodes) == 0: return []
        while True:
            n /= 2
            b_nodes = self.getAreaNodes([pos[0] - n, pos[1] - n], [pos[0] + n, pos[1] + n], a_nodes)
            new_len = len(b_nodes)
            if new_len > 10:
                a_nodes = b_nodes
            elif new_len > 0:
                return b_nodes
            else:
                return a_nodes

    def getNearestNode(self, pos, n = 0.001):
        nearest_nodes = self.getNearestNodes(pos, n)
        if len(nearest_nodes) == 0: return None
        nrst_nd = -1
        min = math.inf
        for nd in nearest_nodes:
            dist = calcLength(self.nodes[nd], pos)
            if dist < min:
                nrst_nd = nd
                min = dist
        return nrst_nd

    def getCoefTrafic(hora):
        trafic_o = 0.6 #dispersion
        trafic_base = math.e/(math.sqrt(2 * math.pi) * trafic_o)
        hora = hora % 12
        coef = -0.5 * ((hora - 7.)/ trafic_o / 2 )**2
        value = trafic_base ** coef
        return value

    def pathDistance(self, root, end, path):
        if path[end] == -1: 
            if root == end: 
                return 0
            return -1
        nd = end
        distance = 0
        while nd != root:   
            distance += self.adj_lst[path[nd]][nd][0]
            nd = path[nd]
        return distance
        
    def pathWeight(self, root, end, path, g):
        if path[end] == -1: 
            if root == end: 
                return 0
            return -1
        nd = end
        distance = 0
        while nd != root:   
            distance += g[path[nd]][nd]
            nd = path[nd]
        return distance
    
    def dijkstra(self, root, end, hour = 7):
        start = time.time()        
        hour_trfc = getCoefTrafic(hour)
        weights = [{} for _ in range(self.n)]
        for nd in range(self.n):
            for adj in self.adj_lst[nd]:
                weights[nd][adj] = self.adj_lst[nd][adj][0] * round(1 + self.adj_lst[nd][adj][1] * hour_trfc, 4)
        visited = [False] * self.n
        path = [-1] * self.n
        g = [math.inf] * self.n
        g[root] = 0
        queue = [root]
        visited[end] = True
        test_nds = set()
        test_nds.add(root)
        t = time.time()
        init_t = t - start
        start = time.time()
        while queue:
            nd = queue.pop(0)
            test_nds.add(nd)
            if not visited[nd]:
                for adj in weights[nd]:
                    _f = g[nd] + weights[nd][adj]
                    if _f < g[adj]:
                        g[adj] = _f
                        path[adj] = nd
                        queue.append(adj)
        t = time.time()
        process_t = t - start
        return path, init_t, process_t

    def aStar(self, root, end, hour = 7):
        start = time.time()
        hour_trfc = getCoefTrafic(hour)
        weights = [{} for _ in range(self.n)]
        end_dt = self.nodes[end]
        path = [-1] * self.n
        g = [math.inf] * self.n
        h = [None] * self.n
        f = [math.inf] * self.n
        g[root] = 0
        test_nds = set()
        nd = root
        test_nds.add(nd)
        t = time.time()
        init_t = t - start
        start = time.time()
        while not(nd == end or nd == -1):
            test_nds.remove(nd)
            for adj in self.adj_lst[nd]:
                if weights[nd].get(adj) is None: weights[nd][adj] = self.adj_lst[nd][adj][0] * round(1 + self.adj_lst[nd][adj][1] * hour_trfc, 4)
                if h[adj] is None: h[adj] = calcLength(self.nodes[adj], end_dt)
                _g = g[nd] + weights[nd][adj]
                if _g < g[adj]:
                    path[adj] = nd
                    g[adj] = _g
                    f[adj] = _g + h[adj]
                    if adj not in test_nds: test_nds.add(adj)
            min = math.inf
            nd = -1
            for tst_nd in test_nds:
                if f[tst_nd] < min:
                    min = f[tst_nd]
                    nd = tst_nd
        t = time.time()
        process_t = t - start
        return path, init_t, process_t

In [8]:
import tkinter
import tkintermapview
from tkinter import ttk

root = tkinter.Tk()
root.pack_propagate(0)
root.wm_title("MyGPS")
root.geometry("1024x768")
root.configure(background='black')

G = Graph()
n_markers = 0
markers = [[None, None], [None, None]]
nodes = [None] * 2
paths = [None] * 2
coords_ttl_lbl = [ttk.Label(root, text="Coordenada Origen", background="black", foreground="white", font=('Roboto', 12, 'bold')),
                  ttk.Label(root, text="Coordenada Destino", background="black", foreground="white", font=('Roboto', 12, 'bold'))]
coords_in = [[tkinter.Entry(root, width=30) for _ in range(2)] for _ in range(2)]
coords_lbls = [[ttk.Label(root, text="Lat", background="black", foreground="white"), ttk.Label(root, text="Lon", background="black", foreground="white")] for _ in range(2)]

_hour_ = 7
hour_str = tkinter.StringVar(value="7")
hour_lbl = ttk.Label(root, text="Hora", background="black", foreground="white", font=('Roboto', 12, 'bold'))
hour_in = tkinter.Entry(root, textvariable=hour_str, width= 5)

map_lbl1 = ttk.Label(root, text="Algoritmo A*", background="black", foreground="white", font=('Roboto', 12, 'bold'))
map_lbl2 = ttk.Label(root, text="Algoritmo de Dijkstra", background="black", foreground="white", font=('Roboto', 12, 'bold'))

map_widget1 = tkintermapview.TkinterMapView(root, width=450, height=450, corner_radius=0)
map_widget1.set_address("Santiago de Surco")
map_widget1.set_zoom(13)

map_widget2 = tkintermapview.TkinterMapView(root, width=450, height=450, corner_radius=0)
map_widget2.set_address("Santiago de Surco")
map_widget2.set_zoom(13)
data_lbls = [ttk.Label(root, text="init(s)=  \nprocess(s)=", background="black", foreground="white") for _ in range(2)]

#Functions
def centerMap():
    map_widget1.set_address("Santiago de Surco")
    map_widget1.set_zoom(13)
    map_widget2.set_address("Santiago de Surco")
    map_widget2.set_zoom(13)
    
def draw_path():
    global nodes, _hour_, data_lbls
    centerMap()
    if None in nodes:
        return
    tmp = _hour_
    try:
        _hour_ = int(hour_in.get())% 24
    except:
        _hour_ = tmp
    print(_hour_, nodes[0], nodes[1])
    hour_in.delete(0, tkinter.END)
    hour_str.set(str(_hour_))
    if None not in paths:
        paths[0].delete()
        paths[1].delete()
    init_t = [None]*2
    process_t = [None]*2
    way = [None]*2
    way[0], init_t[0], process_t[0] = G.getWayCoords(nodes[0], nodes[1], hour=_hour_)
    way[1], init_t[1], process_t[1] = G.getWayCoords2(nodes[0], nodes[1], hour=_hour_)
    paths[0] = map_widget1.set_path(way[0])
    paths[1] = map_widget2.set_path(way[1])
    data_lbls[0].config(text = "init(s)="+str(init_t[0])+" s"+"\nprocess(s)="+str(process_t[0])+" s")
    data_lbls[1].config(text = "init(s)="+str(init_t[1])+" s"+"\nprocess(s)="+str(process_t[1])+" s")

def push_button_path(*args):
    for i in range(2):
        try:
            lat = float(coords_in[i][0].get())
            lon = float(coords_in[i][1].get())
            print(lat, lon)
            if lat != "" and lon != "":
                add_marker([lat, lon], i)
        except:
            lat = lon = 0
    draw_path()

def add_marker(coords, id):
    node = G.getNearestNode(coords, 0.001)
    if node is not None:
        nodes[id] = node
        coords_in[id][0].delete(0, tkinter.END)
        coords_in[id][0].insert(0, str(coords[0]))
        coords_in[id][1].delete(0, tkinter.END)
        coords_in[id][1].insert(0, str(coords[1]))
        if markers[0][id] is not None and markers[1][id] is not None:
            markers[0][id].delete()
            markers[1][id].delete()
        markers[0][id] = map_widget1.set_marker(coords[0], coords[1], text="Mark" + str(id))
        markers[1][id] = map_widget2.set_marker(coords[0], coords[1], text="Mark" + str(id))
        return True
    return False

def add_marker_event(coords):
    global markers, n_markers, _hour_
    print("Add marker:", coords)
    if n_markers < 2:
        if add_marker(coords, n_markers):
            draw_path()
            n_markers = (n_markers + 1) % 2

path_btn = tkinter.Button(text="CAMINO", command=push_button_path)
hour_btn = tkinter.Button() 
map_widget1.add_right_click_menu_command(label="Add Marker", command=add_marker_event, pass_coords=True)
map_widget2.add_right_click_menu_command(label="Add Marker", command=add_marker_event, pass_coords=True)

k = 0
count = 0
for i in range(2):
    coords_ttl_lbl[i].grid(row=i+k, column=0, columnspan=4, pady=4, padx=1)
    k+=1
    coords_lbls[i][0].grid(row=i+k, column=0, sticky = tkinter.W, pady=4, padx=1)
    coords_in[i][0].grid(row=i+k, column=1, sticky = tkinter.W, pady=4, padx=1)
    coords_lbls[i][1].grid(row=i+k, column=2, sticky = tkinter.W, pady=4, padx=1)
    coords_in[i][1].grid(row=i+k, column=3, sticky = tkinter.W, pady=4, padx=1)
    count +=1
count += k

hour_lbl.grid(row=count, column=0,  columnspan=4)
count+=1
hour_in.grid(row=count, column=0,  columnspan=4)
count+=1
path_btn.grid(row=count, column=0,  columnspan=4, pady=4, padx=1)
map_lbl1.place(x=20, y=250)
map_lbl2.place(x=550, y=250)
data_lbls[0].place(x=20, y=730)
data_lbls[1].place(x=550, y=730)
map_widget1.place(x=20, y=280)
map_widget2.place(x=550, y=280)

root.mainloop()

nodes: 3888
